In [1]:
from ast import literal_eval
import numpy as np
import math
import re
import pandas as pd

import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

from difflib import SequenceMatcher
from collections import Counter
import itertools

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

from fuzzywuzzy import fuzz
import nltk

pd.set_option('display.max_columns', 1000)

In [2]:
cd Downloads/fuzzy_matching

/Users/acc48668/Downloads/fuzzy_matching


In [3]:
#jaro version
def sort_token_alphabetically(word):
    token = re.split('[,. ]', word)
    sorted_token = sorted(token)
    return ' '.join(sorted_token)

def get_jaro_distance(first, second, winkler=True, winkler_ajustment=True,
                      scaling=0.1, sort_tokens=True):
    """
    :param first: word to calculate distance for
    :param second: word to calculate distance with
    :param winkler: same as winkler_ajustment
    :param winkler_ajustment: add an adjustment factor to the Jaro of the distance
    :param scaling: scaling factor for the Winkler adjustment
    :return: Jaro distance adjusted (or not)
    """
    if sort_tokens:
        first = sort_token_alphabetically(first)
        second = sort_token_alphabetically(second)

    if not first or not second:
        raise JaroDistanceException(
            "Cannot calculate distance from NoneType ({0}, {1})".format(
                first.__class__.__name__,
                second.__class__.__name__))

    jaro = _score(first, second)
    cl = min(len(_get_prefix(first, second)), 4)

    if all([winkler, winkler_ajustment]):  # 0.1 as scaling factor
        return round((jaro + (scaling * cl * (1.0 - jaro))) * 100.0) / 100.0

    return jaro

def _score(first, second):
    shorter, longer = first.lower(), second.lower()

    if len(first) > len(second):
        longer, shorter = shorter, longer

    m1 = _get_matching_characters(shorter, longer)
    m2 = _get_matching_characters(longer, shorter)

    if len(m1) == 0 or len(m2) == 0:
        return 0.0

    return (float(len(m1)) / len(shorter) +
            float(len(m2)) / len(longer) +
            float(len(m1) - _transpositions(m1, m2)) / len(m1)) / 3.0

def _get_diff_index(first, second):
    if first == second:
        pass

    if not first or not second:
        return 0

    max_len = min(len(first), len(second))
    for i in range(0, max_len):
        if not first[i] == second[i]:
            return i

    return max_len

def _get_prefix(first, second):
    if not first or not second:
        return ""

    index = _get_diff_index(first, second)
    if index == -1:
        return first

    elif index == 0:
        return ""

    else:
        return first[0:index]

def _get_matching_characters(first, second):
    common = []
    limit = math.floor(min(len(first), len(second)) / 2)

    for i, l in enumerate(first):
        left, right = int(max(0, i - limit)), int(
            min(i + limit + 1, len(second)))
        if l in second[left:right]:
            common.append(l)
            second = second[0:second.index(l)] + '*' + second[
                                                       second.index(l) + 1:]

    return ''.join(common)

def _transpositions(first, second):
    return math.floor(
        len([(f, s) for f, s in zip(first, second) if not f == s]) / 2.0)

def get_top_matches(reference, value_list, max_results=None):
    scores = []
    if not max_results:
        max_results = len(value_list)
    for val in value_list:
        score_sorted = get_jaro_distance(reference, val)
        score_unsorted = get_jaro_distance(reference, val, sort_tokens=False)
        scores.append((val, max(score_sorted, score_unsorted)))
    scores.sort(key=lambda x: x[1], reverse=True)

    return scores[:max_results]

class JaroDistanceException(Exception):
    def __init__(self, message):
        super(Exception, self).__init__(message)

In [4]:
#load csv
df = pd.read_csv('bo_attr_lookup_new.csv', encoding='latin1')
df = df.dropna()

#conver child col to list
df['children_org_name_list'] = df['children_org_name_list'].map(lambda x: [x])

#jaro score
df['jaro_func_score'] = df[['parent_org_name', 'children_org_name_list']].agg(lambda x: get_top_matches(*x), axis=1)
df['jaro_func_score'] = df.jaro_func_score.apply(lambda x: [val[1] for val in x])
df['jaro_func_score'] = df.jaro_func_score.apply(lambda x: ', '.join([str(i) for i in x])).astype(float)

#create df of just medium and high
df1 = df[df['match_confidence'].isin(['MEDIUM','HIGH'])]

#create low df
df2 = df[df['match_confidence'] == 'LOW']

#create sample of 2k rows of low df
df2 = df2.sample(2000)

#concat dfs
frames = [df1,df2]
df = pd.concat(frames)

df.head()

,parent_org_name,children_org_name_list,match_confidence_score,match_confidence,jaro_func_score
3,+MARTINS,[SAN MARINA],0.750000,MEDIUM,0.78
8,111 SOUTH,[1&1 IONOS],0.769231,MEDIUM,0.67
13,247 CUSTOMER PVT. LTD.,[24/7 CUSTOMER PRIVATE LIMITED],0.843750,MEDIUM,0.89
18,3M,[3M],1.000000,HIGH,1.00
57,A. K. ENTERPRISE,[ROUSH ENTERPRISE],0.812500,MEDIUM,0.79


In [5]:
def total_chars(row):

    results0 = []
    results1 = []
    results2 = []
    results3 = []
    results4 = []
    results5 = []
    
    p = row['parent_org_name'].lower()
    for i in row['children_org_name_list']:
        
        t = i.lower()
        
        dict1 = Counter(p)
        dict2 = Counter(t)
        
        commonDict = dict1 & dict2
        allChars = len(dict1) + len(dict2)
        childCharsPositions = len(t)
        
        results0.append((i, childCharsPositions)[1]) #totalCharacterPositions
        results1.append((i, SequenceMatcher(None, t, p).find_longest_match(0, len(t), 0, len(p))[2])[1]) #consecutiveMatchingChars
        results2.append((i, round(SequenceMatcher(None, t, p).ratio(),2))[1]) #sequence_matcher_ratio
        results3.append((i, sum(int(k==v) for k,v in zip(t, p)))[1]) #countSamePosition
        results4.append((i, allChars)[1])#countTotalUniqueChars
        results5.append((i, len(commonDict))[1]) #countCommonChars

    return pd.Series([results0, results1, results2, results3, results4, results5])

df[['total_chars_positions'
    , 'consecutive_chars'
    , 'sequence_matcher_ratio'
    , 'count_same_position'
    , 'count_total_unique_chars'
    , 'count_common_chars'
   ]]  = df.apply(total_chars, axis=1)

df['total_chars_positions'] = df.total_chars_positions.apply(lambda x: ', '.join([str(i) for i in x])).astype(float)
df['consecutive_chars'] = df.consecutive_chars.apply(lambda x: ', '.join([str(i) for i in x])).astype(float)
df['sequence_matcher_ratio'] = df.sequence_matcher_ratio.apply(lambda x: ', '.join([str(i) for i in x])).astype(float)
df['count_same_position'] = df.count_same_position.apply(lambda x: ', '.join([str(i) for i in x])).astype(float)
df['count_total_unique_chars'] = df.count_total_unique_chars.apply(lambda x: ', '.join([str(i) for i in x])).astype(float)
df['count_common_chars'] = df.count_common_chars.apply(lambda x: ', '.join([str(i) for i in x])).astype(float)

df['percent_matched'] = round(df['count_common_chars'] / df['count_total_unique_chars'],2)
df['percent_same_position'] = round(df['count_same_position'] / df['total_chars_positions'],2)

df.head(10)

,parent_org_name,children_org_name_list,match_confidence_score,match_confidence,jaro_func_score,total_chars_positions,consecutive_chars,sequence_matcher_ratio,count_same_position,count_total_unique_chars,count_common_chars,percent_matched,percent_same_position
3,+MARTINS,[SAN MARINA],0.750000,MEDIUM,0.78,10.0,3.0,0.56,0.0,15.0,6.0,0.40,0.00
8,111 SOUTH,[1&1 IONOS],0.769231,MEDIUM,0.67,9.0,2.0,0.44,4.0,14.0,4.0,0.29,0.44
13,247 CUSTOMER PVT. LTD.,[24/7 CUSTOMER PRIVATE LIMITED],0.843750,MEDIUM,0.89,29.0,12.0,0.78,3.0,36.0,16.0,0.44,0.10
18,3M,[3M],1.000000,HIGH,1.00,2.0,2.0,1.00,2.0,4.0,2.0,0.50,1.00
57,A. K. ENTERPRISE,[ROUSH ENTERPRISE],0.812500,MEDIUM,0.79,16.0,11.0,0.69,11.0,22.0,8.0,0.36,0.69
58,A.B. EDWARDS ENTERPRISES,[A. G. EDWARDS & SONS],0.769231,MEDIUM,0.73,20.0,10.0,0.64,2.0,25.0,9.0,0.36,0.10
70,AAP CO. LTD.,[AAP LEHRERFACHVERLAGE],0.900000,HIGH,0.73,21.0,4.0,0.30,4.0,20.0,5.0,0.25,0.19
87,ABB,[ABBVIE],0.750000,MEDIUM,0.88,6.0,3.0,0.67,3.0,7.0,2.0,0.29,0.50
94,ABBVIE,[ABBVIE],1.000000,HIGH,1.00,6.0,6.0,1.00,6.0,10.0,5.0,0.50,1.00
111,ABERCROMBIE FITCH,[ABERCROMBIE & FITCH],1.000000,HIGH,0.98,19.0,12.0,0.94,12.0,25.0,12.0,0.48,0.63


In [6]:
#trigram function
def split_str(s, N):
    rmax = np.clip(len(s)-N, a_min=0, a_max=None)
    return [s[0+i:N+i] for i in range(0, rmax+1)]

#arrange dfs and cols
df = df[['parent_org_name','children_org_name_list','jaro_func_score','sequence_matcher_ratio']]
df1 = df[['parent_org_name','jaro_func_score','sequence_matcher_ratio']]
df['parent_trigram'] = df['parent_org_name'].apply(lambda x: split_str(x, 3))
# del df[['jaro_func_score','sequence_matcher_ratio']]

#binarizer option
mlb = MultiLabelBinarizer(sparse_output=True)

mlb_df = df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df.pop('parent_trigram')),
                index=df.index,
                columns=mlb.classes_))

#create numbers as column names
mlb_df = mlb_df.rename(columns={x:y for x,y in zip(mlb_df.columns,range(0,len(mlb_df.columns)))})

#convert 0,1s to col name
values = np.where(mlb_df.eq(1), mlb_df.columns, mlb_df)
mlb_df = pd.DataFrame(values, columns=mlb_df.columns)

#concat all columns into a list, agg per row, and drop zeros
mlb_df['ohe_locations'] = (mlb_df.iloc[:,3:].astype(str)
 .agg(lambda x: ' '.join(i for i in x if i != '0'), axis=1)
 .str.split()
)

#create new db of relevant cols
mlb_df = mlb_df[[0,1,'ohe_locations']]

#change column names
mlb_df.columns = [['parent_org_name','children_org_name_list','ohe_locations']]

#add back in the jaro score from df1
jaro_func_score = list(df1['jaro_func_score'])
sequence_matcher_ratio = list(df1['sequence_matcher_ratio'])
mlb_df['jaro_func_score'] = jaro_func_score
mlb_df['sequence_matcher_ratio'] = sequence_matcher_ratio

mlb_df.head()

,parent_org_name,children_org_name_list,ohe_locations,jaro_func_score,sequence_matcher_ratio
0,+MARTINS,[SAN MARINA],"[0.56, 307, 932, 2823, 3509, 4852, 5348]",0.78,0.56
1,111 SOUTH,[1&1 IONOS],"[0.44, 236, 497, 506, 508, 4325, 5109, 5687]",0.67,0.44
2,247 CUSTOMER PVT. LTD.,[24/7 CUSTOMER PRIVATE LIMITED],"[0.78, 81, 172, 215, 424, 537, 569, 593, 1438,...",0.89,0.78
3,3M,[3M],"[3, 555]",1.00,1.00
4,A. K. ENTERPRISE,[ROUSH ENTERPRISE],"[0.69, 98, 156, 417, 423, 658, 1923, 1975, 288...",0.79,0.69


In [8]:
#create train test split
X = mlb_df[mlb_df.columns[3:]].values
y = mlb_df[mlb_df.columns[0]].values

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3)

In [9]:
#classification report function and classifiers
def pandas_classification_report(y_true, y_pred):
    metrics_summary = precision_recall_fscore_support(
        y_true=y_test
        , y_pred=y_predictions
        , labels=np.unique(y_pred)
    )

    avg = list(precision_recall_fscore_support(
            y_true=y_test, 
            y_pred=y_predictions,
            average='weighted'))

    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support']
    class_report_df = pd.DataFrame(
        list(metrics_summary),
        index=metrics_sum_index)

    support = class_report_df.loc['support']
    total = support.sum() 
    avg[-1] = total

    class_report_df['avg / total'] = avg

    return class_report_df.T

classifiers = {
    "Naive Bayes": GaussianNB(),
    "LogisiticRegression": LogisticRegression(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "KNN": KNeighborsClassifier(),
    "Support Vector Classifier": SVC(),
}

In [ ]:
import warnings
warnings.filterwarnings('always')

f, axes = plt.subplots(1, 6, figsize=(20, 5), sharey='row')

print('Classification Report: \n')

for i, (key, classifier) in enumerate(classifiers.items()):
    y_pred = classifier.fit(X_train, y_train).predict(X_test)
    y_predictions = classifier.predict(X_test)
    df_class_report = pandas_classification_report(y_true=y_test, y_pred=y_predictions)
    print((classifier),'\n',df_class_report)
    cf_matrix = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(cf_matrix)
#                                   display_labels=)
    disp.plot(ax=axes[i], xticks_rotation=0)
    disp.ax_.set_title(key)
    disp.im_.colorbar.remove()
    disp.ax_.set_xlabel('')
    if i!=0:
        disp.ax_.set_ylabel('')
        
f.text(0.4, 0.1, 'Predicted label', ha='left')
plt.subplots_adjust(wspace=0.40, hspace=0.1)

f.colorbar(disp.im_, ax=axes)
plt.show()

Classification Report: 



/Users/acc48668/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/acc48668/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/acc48668/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


GaussianNB() 
              precision  recall  f1-score  support
0                  0.0     0.0       0.0      0.0
1                  0.0     0.0       0.0      0.0
2                  0.0     0.0       0.0      0.0
3                  0.0     0.0       0.0      0.0
4                  0.0     0.0       0.0      0.0
...                ...     ...       ...      ...
729                0.0     0.0       0.0      0.0
730                0.0     0.0       0.0      0.0
731                0.0     0.0       0.0      0.0
732                0.0     0.0       0.0      0.0
avg / total        0.0     0.0       0.0      0.0

[734 rows x 4 columns]


/Users/acc48668/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/acc48668/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/acc48668/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


LogisticRegression() 
              precision  recall  f1-score  support
0                  0.0     0.0       0.0      0.0
1                  0.0     0.0       0.0      0.0
2                  0.0     0.0       0.0      0.0
3                  0.0     0.0       0.0      0.0
4                  0.0     0.0       0.0      0.0
5                  0.0     0.0       0.0      0.0
6                  0.0     0.0       0.0      0.0
7                  0.0     0.0       0.0      0.0
8                  0.0     0.0       0.0      0.0
9                  0.0     0.0       0.0      0.0
10                 0.0     0.0       0.0      0.0
11                 0.0     0.0       0.0      0.0
12                 0.0     0.0       0.0      0.0
13                 0.0     0.0       0.0      0.0
14                 0.0     0.0       0.0      0.0
15                 0.0     0.0       0.0      0.0
16                 0.0     0.0       0.0      0.0
17                 0.0     0.0       0.0      0.0
avg / total        0.0     

/Users/acc48668/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/acc48668/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/acc48668/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


DecisionTreeClassifier() 
              precision  recall  f1-score  support
0                  0.0     0.0       0.0      0.0
1                  0.0     0.0       0.0      0.0
2                  0.0     0.0       0.0      0.0
3                  0.0     0.0       0.0      0.0
4                  0.0     0.0       0.0      0.0
...                ...     ...       ...      ...
721                0.0     0.0       0.0      0.0
722                0.0     0.0       0.0      0.0
723                0.0     0.0       0.0      0.0
724                0.0     0.0       0.0      0.0
avg / total        0.0     0.0       0.0      0.0

[726 rows x 4 columns]


# Parent df stuff - count unique items in list

In [42]:
parent_df = pd.DataFrame(' '.join(df['parent_org_name'].tolist()), columns=['parent_org_name'], index=[0])
parent_df['parent_trigram'] = parent_df['parent_org_name'].apply(lambda x: split_str(x, 3))
parent_df

,parent_org_name,parent_trigram
0,+MARTINS 111 SOUTH 247 CUSTOMER PVT. LTD. 3M A...,"[+MA, MAR, ART, RTI, TIN, INS, NS , S 1, 11, ..."


In [43]:
parent_df = parent_df[['parent_trigram']]

,parent_trigram
0,"[+MA, MAR, ART, RTI, TIN, INS, NS , S 1, 11, ..."


In [61]:
#number of trigrams
parent_df.parent_trigram.str.len().sum()

110150

In [66]:
#number of unique trigrams
parent_explode_df = parent_df['parent_trigram'].explode()
parent_explode_df = pd.DataFrame(parent_explode_df)

,parent_trigram
0,+MA
0,MAR
0,ART
0,RTI
0,TIN
...,...
0,IRA
0,RA
0,A S
0,S


In [67]:
parent_explode_df.parent_trigram.nunique()

6473

In [41]:
parent_list = parent_df.parent_trigram.tolist()
perent_list

NameError: name 'perent_list' is not defined

# Test Code

In [28]:
parent_list = list(df.parent_org_name)

def find_ngrams(input_list, n):
    return zip(*[input_list[i:] for i in range(n)])

In [ ]:
#factorize option (fastest)
v = df.parent_trigram.values
l = [len(x) for x in v.tolist()]
f, u = pd.factorize(np.concatenate(v))
n, m = len(v), u.size
i = np.arange(n).repeat(l)

dummies = pd.DataFrame(
    np.bincount(i * m + f, minlength=n * m).reshape(n, m),
    df.index, u
)

df.drop('parent_trigram', 1).join(dummies)

In [ ]:
#get dummies option
df.drop('parent_trigram', 1).join(df.parent_trigram.str.join('|').str.get_dummies())

In [ ]:
#put all 100k list trigrams on y axis
ohe_df = pd.DataFrame(data=df, columns=parent_list)
ohe_df.head()

In [ ]:
#attempts to convert 0,1s to column name
mlb_df.loc[:,1:]  * mlb_df.columns[1:].astype(int)
for col in mlb_df.columns:
    mlb_df.loc[mlb_df[col] == 1, col] = col

In [ ]:
#create df with all trigrams as columns

df1 = df[['parent_org_name']]

ohe_df = pd.concat([df1, ohe_df], axis=1, sort=False)
ohe_df.head()

In [ ]:
# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')

# passing bridge-types-cat column (label encoded values of bridge_types)
enc_df = pd.DataFrame(enc.fit_transform(df[['parent_org_name']]).toarray())

# merge with main df bridge_df on key values
df1 = df.join(enc_df)
df1.head()

In [ ]:
types = df.applymap(type)
types

In [29]:
find_ngrams(parent_list, 3)

In [30]:
input_list = ['all', 'this', 'happened', 'more', 'or', 'less']

keywords = [''.join(i) for i in itertools.product(input_list, repeat = 3)]
vector = CountVectorizer(analyzer='char', ngram_range=(3,3), vocabulary=keywords)
tr_test = vector.transform(['word1'])
print(tr_test)
print(vector)


CountVectorizer(analyzer='char', ngram_range=(3, 3),
                vocabulary=['allallall', 'allallthis', 'allallhappened',
                            'allallmore', 'allallor', 'allallless',
                            'allthisall', 'allthisthis', 'allthishappened',
                            'allthismore', 'allthisor', 'allthisless',
                            'allhappenedall', 'allhappenedthis',
                            'allhappenedhappened', 'allhappenedmore',
                            'allhappenedor', 'allhappenedless', 'allmoreall',
                            'allmorethis', 'allmorehappened', 'allmoremore',
                            'allmoreor', 'allmoreless', 'allorall', 'allorthis',
                            'allorhappened', 'allormore', 'alloror',
                            'allorless', ...])


In [ ]:
ohe_df['parent_org_name'] = ohe_df.parent_org_name.astype(str)

for c in ohe_df['parent_org_name']:
    ohe_df[c] = ohe_df['parent_org_name'].apply(lambda x: int(c in x))
    print (ohe_df)

In [34]:
#trigram function
def split_str(s, N):
    rmax = np.clip(len(s)-N, a_min=0, a_max=None)
    return [s[0+i:N+i] for i in range(0, rmax+1)]

df['parent_org_name'].apply(lambda x: split_str(x, 3))
# input_list.apply(lambda x: split_str(x, 3))

3                           [+MA, MAR, ART, RTI, TIN, INS]
8                      [111, 11 , 1 S,  SO, SOU, OUT, UTH]
13       [247, 47 , 7 C,  CU, CUS, UST, STO, TOM, OME, ...
18                                                    [3M]
57       [A. , . K,  K., K. , . E,  EN, ENT, NTE, TER, ...
                               ...                        
10303    [NAT, ATO, TOM, OMA, MAS, AS , S S,  SC, SCH, ...
11965    [RAD, ADI, DIU, IUS, US , S H,  HE, HEA, EAL, ...
809      [AME, MER, ERI, RIC, ICA, CAN, AN , N M,  MU, ...
840      [AMI, MIX, IXC, XCO, COM, OM , M I,  IN, INC, ...
4533                                  [DEV, EVI, VID, IDA]
Name: parent_org_name, Length: 5210, dtype: object

In [15]:
def ngrams(input, n):
    input = input.split(' ')
    output = []
    for i in range(len(input)-n+1):
        output.append(input[i:i+n])
    return output

In [17]:
ngrams(df.parent_org_name, 3)

AttributeError: 'Series' object has no attribute 'split'

In [18]:
def find_ngrams(input_list, n):
    return zip(*[input_list[i:] for i in range(n)])

In [20]:
find_ngrams(input_list,1)

In [14]:
words = nltk.word_tokenize('this is a string')
my_bigrams = nltk.bigrams(words)
my_trigrams = nltk.trigrams(words)

In [20]:
my_bigrams

<generator object bigrams at 0x7fe9a7255890>

In [16]:
from collections import deque
from itertools import islice

def ngrams(message, n=1):
    it = iter(message.split())
    window = deque(islice(it, n), maxlen=n)
    yield tuple(window)
    for item in it:
        window.append(item)
        yield tuple(window)

In [17]:
ngrams(input_list,3)

<generator object ngrams at 0x7fe9a7255120>

In [21]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
text = "Hi How are you? i am fine and you"
token=nltk.word_tokenize(text)
bigrams=ngrams(token,2)

In [22]:
bigrams

<generator object ngrams at 0x7fe9a7278ac0>

In [44]:
parent_list = str(df.parent_org_name)
parent_list

'3                                  +MARTINS\n8                                 111 SOUTH\n13                   247 CUSTOMER PVT. LTD.\n18                                       3M\n57                         A. K. ENTERPRISE\n                        ...                \n10303               NATOMAS SCHOOL DISTRICT\n11965                     RADIUS HEALTH INC\n809      AMERICAN MUSEUM OF NATURAL HISTORY\n840                            AMIXCOM INC.\n4533                                 DEVIDA\nName: parent_org_name, Length: 5210, dtype: object'

In [46]:
parent_org_name_string = pd.DataFrame(' '.join(df['parent_org_name'].tolist()), columns=['parent_org_name'], index=[0])

In [51]:
parent_org_name_string = pd.DataFrame(' '.join(df['parent_org_name'].tolist()), columns=['parent_org_name'], index=[0])
parent_org_name_string['parent_trigram'] = parent_org_name_string['parent_org_name'].apply(lambda x: split_str(x, 3))
parent_org_name_string

,parent_org_name,parent_trigram
0,+MARTINS 111 SOUTH 247 CUSTOMER PVT. LTD. 3M A...,"[+MA, MAR, ART, RTI, TIN, INS, NS , S 1, 11, ..."


In [ ]:
print('original df shape: \n', df.shape)
print('')

print('dtypes: \n', df.dtypes)
print('')

print('original match confidence bins values counts: \n', df.match_confidence.value_counts())
print('')

#create df of just medium and high
df1 = df[df['match_confidence'].isin(['MEDIUM','HIGH'])]
print('high low df shape: \n', df1.shape)
print('high and low df value counts: \n', df1.match_confidence.value_counts())
print('')

#create low df
df2 = df[df['match_confidence'] == 'LOW']
print('low df shape: \n', df2.shape)
print('')

#create sample of 2k rows of low df
df2 = df2.sample(2000)

#concat dfs
frames = [df1,df2]
df = pd.concat(frames)
print('new df shape with sample of 2k low rows: \n', df.shape)
print('new df value counts: \n', df.match_confidence.value_counts())
print(df.head())